In [ ]:
#Ex-6 CNN
# =========================================================
# BCCD Dataset Image Classification with Augmentation
# =========================================================

# ------------------------------
# a. Load the dataset
# ------------------------------
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Path to dataset (adjust according to your local setup)
dataset_path = "BCCD"  # folder containing images

# Image dimensions
img_width, img_height = 128, 128

# Class labels
classes = ["WBC", "RBC", "Platelets"]

# Load images and labels
images = []
labels = []

for label, class_name in enumerate(classes):
    class_path = os.path.join(dataset_path, class_name)
    for file in os.listdir(class_path):
        img = load_img(os.path.join(class_path, file), target_size=(img_width, img_height))
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(label)

images = np.array(images, dtype='float32')
labels = np.array(labels)

# ------------------------------
# b. Show the number of testing and training images
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)
print(f"Number of training images: {len(X_train)}")
print(f"Number of testing images: {len(X_test)}")

# ------------------------------
# c. Plot some images
# ------------------------------
plt.figure(figsize=(10,5))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[i].astype('uint8'))
    plt.title(classes[y_train[i]])
    plt.axis('off')
plt.show()

# ------------------------------
# d. Image augmentation – contrast, flipping, rotation
# ------------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7,1.3]
)

test_datagen = ImageDataGenerator(rescale=1./255)

# ------------------------------
# e. After augmentation, show the number of training images
# ------------------------------
# Note: ImageDataGenerator generates batches on the fly; original count stays same
print(f"Training images after augmentation (original count): {len(X_train)}")
print(f"Testing images (rescaled only): {len(X_test)}")

# ------------------------------
# f. Normalizing the training data
# ------------------------------
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)

# ------------------------------
# g. Build a convolutional neural network to train images
# ------------------------------
def build_cnn():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(img_width,img_height,3)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_cnn()

# ------------------------------
# h. Show the training and testing accuracy (before augmentation)
# ------------------------------
history = model.fit(X_train, y_train_cat, epochs=15, batch_size=32, validation_data=(X_test, y_test_cat))

# Plot accuracy
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='test_accuracy')
plt.title("Training and Testing Accuracy (Before Augmentation)")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# ------------------------------
# i. Train CNN with augmentation
# ------------------------------
batch_size = 32
train_generator = train_datagen.flow(X_train, y_train_cat, batch_size=batch_size)
test_generator = test_datagen.flow(X_test, y_test_cat, batch_size=batch_size)

model_aug = build_cnn()

history_aug = model_aug.fit(train_generator, epochs=15, validation_data=test_generator)

# ------------------------------
# j. Show the training and testing accuracy (after augmentation)
# ------------------------------
plt.plot(history_aug.history['accuracy'], label='train_accuracy_aug')
plt.plot(history_aug.history['val_accuracy'], label='test_accuracy_aug')
plt.title("Training and Testing Accuracy (After Augmentation)")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# ------------------------------
# k. Compare accuracy before and after augmentation
# ------------------------------
print("Final Accuracy Comparison:")
print(f"Train Accuracy before augmentation: {history.history['accuracy'][-1]:.4f}")
print(f"Test Accuracy before augmentation: {history.history['val_accuracy'][-1]:.4f}")
print(f"Train Accuracy after augmentation: {history_aug.history['accuracy'][-1]:.4f}")
print(f"Test Accuracy after augmentation: {history_aug.history['val_accuracy'][-1]:.4f}")
